In [1]:
from pandas import DataFrame
from pyarrow import Table
from pyarrow.flight import connect, FlightDescriptor
from random import random

client = connect("grpc://0.0.0.0:8080")

In [2]:
for i in client.list_flights():
    print(i)

<pyarrow.flight.FlightInfo schema=x: double
y: double descriptor=<pyarrow.flight.FlightDescriptor path=[b'linear_regression']> endpoints=[<pyarrow.flight.FlightEndpoint ticket=<pyarrow.flight.Ticket ticket=b'linear_regression'> locations=[<pyarrow.flight.Location b'grpc://0.0.0.0:8080'>]>] total_records=-1 total_bytes=-1>


In [3]:
data = DataFrame([
    {"x": 0.0, "y": 1.0},
    {"x": 3.0, "y": 6.0},
    {"x": 5.0, "y": 11.0},
    {"x": 10.0, "y": 21.0},
])

table = Table.from_pandas(data)

writer, reader = client.do_exchange(FlightDescriptor.for_path("linear_regression"))

writer.begin(table.schema)
writer.write_table(table)
writer.done_writing()


data = reader.read_pandas()

data


,x,y,predict
0,0.0,1.0,0.622642
1,3.0,6.0,6.707547
2,5.0,11.0,10.764151
3,10.0,21.0,20.905660


In [4]:
client.close()